In [20]:
# # install AMPL
# # Install Python API for AMPL:
# $ python -m pip install amplpy --upgrade

# # Install solver modules:
# $ python -m amplpy.modules install highs

# # Activate your AMPL CE license: ............................. (paste id license)
# $ python -m amplpy.modules activate

In [52]:
from amplpy import AMPL, tools
# ampl = tools.ampl_notebook(
#     modules=["highs", "cbc"], # modules to install
#     license_uuid="d3af9008-221f-4220-a118-625786b1fe84")

ampl = AMPL() # instantiate AMPL object

Instance 01

In [118]:
# obj = 14
# [| 13 | 14 |]

m = 2
l = [15, 10]

n = 6
s = [3, 2, 6, 5, 4, 4]

D = [ 
	0, 3, 4, 5, 6, 6, 2,
	3, 0, 1, 4, 5, 7, 3,
	4, 1, 0, 5, 6, 6, 4,
	4, 4, 5, 0, 3, 3, 2,
	6, 7, 8, 3, 0, 2, 4,
	6, 7, 8, 3, 2, 0, 4,
	2, 3, 4, 3, 4, 4, 0,
]

# Model

In [124]:
ampl.eval(r"""
    reset;

    ## VARIABLES
    param m;
    param n;
    set COURIERS := {1..m}; # couriers with loads
    set ITEMS := {1..n}; # items with sizes
    set D_SIZE := {1..n+1};

    param load {COURIERS} > 0 integer;
    param size {ITEMS} > 0 integer;
    param D {D_SIZE, D_SIZE} >= 0 integer; # matrix of distances


    var A {COURIERS, ITEMS} binary;
    var O {COURIERS, ITEMS} >= 0 integer;
    var tot_dist {COURIERS} >= 0 integer; # distance traveled by each courier

    ## OBJECTIVE FUNCTION
    minimize Obj_function:  max {i in COURIERS} tot_dist[i];

    ## CONSTRAINTS
    # constraints to create A
    s.t. A_1 {i in COURIERS}:
        0 < sum {j in ITEMS} (if A[i,j]==1 then size[j] else 0) <= load[i]; # each courier respects its own load capacity
                                                                            # each courier transports at least one item
    s.t. A_2 {j in ITEMS}:
        sum {i in COURIERS} A[i,j] == 1; # each item is transported by one courier
    
    # simmetry breaking
    # TODO

    # # constraints to create O
    # s.t. O_1 {i in COURIERS, j in ITEMS}:
    #     (if A[i, j] == 0 then O[i, j] == 0) &&
    #     (if A[i, j] > 0 then O[i, j] > 0); # each value is in line with those in A
    # #s.t. O_2 {i in COURIERS}:

""")


	line 26 offset 712
	double inequality must be ... <= ... <= ... or ... >= ... >= ...
	context:  0 < sum {j in ITEMS} (if A[i,j]==1 then size[j] else 0) <=  >>> load[i]; <<<  # each courier respects its own load capacity
	
	Please report any bugs at: https://github.com/ampl/amplpy
	
	For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>
	


## Load the data

In [125]:
ampl.param["m"] = m
ampl.param["n"] = n

ampl.param["load"] = l
ampl.param["size"] = s
ampl.param["D"] = D

## Solve with HiGHS

In [126]:
# Specify the solver to use (e.g., HiGHS)
ampl.option["solver"] = "highs"
# Solve
ampl.solve()
# Stop if the model was not solved
assert ampl.get_value("solve_result") == "solved"
# Get objective entity by AMPL name
totalcost = ampl.get_objective('Obj_function')
# Print it
print("Objective is:", totalcost.value())

HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 0
0 simplex iterations
0 branching nodes
Objective is: 0.0


## Solve with COIN-BC

In [127]:
# Specify the solver to use (e.g., HiGHS)
ampl.option["solver"] = "cbc"
# Solve
ampl.solve()
# Stop if the model was not solved
assert ampl.get_value("solve_result") == "solved"
# Get objective entity by AMPL name
totalcost = ampl.get_objective('Obj_function')
# Print it
print("Objective is:", totalcost.value())

cbc 2.10.7cbc 2.10.7: optimal solution; objective 0
0 simplex iterations
Objective is: 0.0
